<a href="https://colab.research.google.com/github/apragupta/ML1_Final_Proj/blob/main/FP_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive, auth
import pandas as pd
import math
import seaborn as sns
# Mount Google Drive - This will prompt you to click a link to authorize
drive.mount('/content/drive')

# Navigate to directory and list files
os.chdir('drive/My Drive/Year 2 NEU/Spring 2021/Machine Learning 1/Final Project')
os.listdir('.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['images_filtered.zip',
 'FP.ipynb',
 'cleaned_filtered.csv',
 'checkpoints',
 'Image_Labels_Subset_Train_GCC-Labels-training (1).tsv',
 'images_filtered',
 'senticap_dataset.json',
 'senticap.json',
 'val2014',
 'senti_cap_sent.csv',
 'FP_1.ipynb',
 'sent_added',
 'FP_2.ipynb',
 'feature_set',
 'FP_3.ipynb']

In [ ]:
import tensorflow as tf

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import collections
import random
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
from tqdm import tqdm

In [ ]:
feature_set = pd.read_csv('feature_set', index_col=0)

In [ ]:
feature_set

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,sentiment
0,1.825437e-07,3.081069e-06,6.151669e-07,6.222912e-07,3.725395e-07,3.865697e-05,6.937968e-08,1.620700e-05,3.596125e-06,1.337872e-07,8.843734e-05,4.655684e-06,1.832618e-06,5.039533e-06,9.255446e-06,2.246210e-06,7.294258e-04,9.038307e-05,4.110755e-06,9.945628e-05,6.501691e-05,1.951231e-05,9.060253e-07,2.130383e-06,1.092555e-06,4.047858e-06,6.023028e-05,3.098373e-05,6.033677e-06,5.805536e-06,1.708504e-06,2.678744e-06,2.970520e-05,1.844297e-07,1.559572e-06,4.176057e-07,1.100943e-06,1.938812e-07,6.542270e-05,7.224590e-08,...,2.901232e-05,1.940366e-05,1.165480e-07,3.043390e-05,2.661321e-06,2.040870e-04,6.865062e-04,6.120695e-05,5.304710e-03,1.058960e-04,1.000266e-05,4.066160e-05,2.502704e-04,5.857625e-06,5.890674e-05,1.585992e-07,9.963989e-07,3.995418e-04,6.530394e-07,5.081920e-05,8.371006e-04,5.926265e-06,4.647442e-06,5.058517e-05,5.015725e-05,8.258094e-04,1.392520e-05,7.575899e-06,3.018130e-06,1.254096e-06,5.359753e-06,3.610757e-06,9.109829e-07,3.424403e-05,2.785006e-04,3.497414e-04,4.637303e-06,3.418282e-05,0.000147,POSITIVE
1,9.743706e-10,3.255081e-07,7.856821e-08,2.840356e-09,3.959591e-07,4.443392e-09,2.565245e-09,5.025527e-08,3.903430e-09,2.469008e-07,4.924568e-07,4.365052e-07,8.465611e-09,7.647478e-08,6.199613e-09,7.457772e-07,1.585488e-07,1.849024e-08,1.053838e-07,8.831719e-09,3.642966e-09,2.966703e-09,1.952014e-07,1.680321e-07,2.713867e-08,9.944913e-09,6.538125e-09,1.491354e-09,9.956842e-07,2.976480e-09,1.488291e-08,1.188868e-08,2.683461e-09,6.199190e-08,3.418475e-08,1.018170e-07,1.676259e-07,1.886993e-08,2.754686e-08,1.156055e-08,...,1.680018e-06,2.198997e-09,4.194494e-08,2.478991e-09,7.773210e-09,6.077891e-06,8.932924e-06,7.445165e-07,7.559190e-08,4.021039e-08,6.701115e-06,4.426876e-07,3.854181e-07,1.219906e-07,1.240589e-07,4.509332e-08,2.188769e-06,1.066551e-06,4.517138e-09,1.519828e-06,6.997589e-01,2.334237e-05,1.053775e-05,7.682678e-09,1.164126e-07,1.551132e-09,9.094554e-08,1.917048e-09,2.982752e-07,4.593377e-09,2.317335e-08,7.678064e-09,2.791685e-08,4.743430e-08,1.611797e-08,6.253970e-10,1.991041e-08,1.679559e-08,0.000002,POSITIVE
2,9.597818e-09,5.120547e-06,1.261530e-06,6.598006e-07,1.151605e-07,5.274585e-07,1.298079e-07,4.675381e-09,2.819261e-08,8.169543e-08,1.083971e-06,3.002534e-06,3.093216e-07,1.442714e-07,1.366039e-07,4.623658e-07,1.402487e-06,3.970051e-08,1.132931e-08,3.031383e-08,1.816143e-07,2.663476e-08,8.411089e-07,3.151740e-06,2.916771e-08,1.629456e-07,2.073944e-07,3.961776e-08,1.356191e-07,7.080595e-08,8.565229e-07,4.058273e-06,1.896106e-07,7.613246e-06,3.898921e-07,4.763010e-07,3.934424e-07,3.019887e-08,6.950574e-08,1.654743e-08,...,3.827929e-08,2.186444e-08,6.628717e-08,9.925559e-08,2.112798e-08,1.428263e-07,8.316859e-08,3.577851e-07,3.503696e-07,6.865386e-06,4.277629e-05,4.910936e-06,1.148648e-05,3.399977e-08,8.006339e-05,4.686343e-06,2.362347e-05,6.000756e-05,1.490726e-05,5.801792e-06,1.305673e-02,2.948562e-06,9.668110e-06,2.451997e-06,3.147012e-06,8.116123e-06,4.312099e-07,8.462425e-08,9.509935e-08,3.647951e-07,3.106075e-07,9.817934e-08,3.188724e-08,3.403931e-07,1.808005e-07,1.989944e-07,4.030477e-07,1.957145e-07,0.000001,POSITIVE
3,3.576900e-06,1.137077e-06,3.392912e-06,4.956223e-07,3.598596e-06,2.840574e-05,8.567486e-06,1.668360e-06,1.695566e-06,1.311694e-05,5.366268e-07,2.751898e-07,4.683070e-07,2.717718e-06,9.243219e-07,1.011731e-06,3.075690e-07,7.056495e-06,4.372628e-06,8.791137e-07,2.794894e-06,2.506775e-06,2.026819e-05,1.226766e-05,3.039562e-05,2.121414e-07,1.506391e-07,1.143929e-06,3.671373e-07,5.045163e-07,7.446660e-07,1.117263e-07,2.205233e-07,1.883190e-05,2.240813e-05,1.784171e-06,4.505926e-06,2.137570e-06,1.327328e-07,4.961411e-07,...,1.099388e-06,9.186014e-08,2.559548e-07,6.368066e-08,1.925686e-06,1.072664e-04,1

In [ ]:
import sklearn

# Feature Selection

In [ ]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.preprocessing import OneHotEncoder
import random

In [ ]:
feature_set['sentiment'] = feature_set['sentiment'].apply(lambda x: 0 if x == 'POSITIVE' else 1)

In [ ]:
feature_set

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,sentiment
0,1.825437e-07,3.081069e-06,6.151669e-07,6.222912e-07,3.725395e-07,3.865697e-05,6.937968e-08,1.620700e-05,3.596125e-06,1.337872e-07,8.843734e-05,4.655684e-06,1.832618e-06,5.039533e-06,9.255446e-06,2.246210e-06,7.294258e-04,9.038307e-05,4.110755e-06,9.945628e-05,6.501691e-05,1.951231e-05,9.060253e-07,2.130383e-06,1.092555e-06,4.047858e-06,6.023028e-05,3.098373e-05,6.033677e-06,5.805536e-06,1.708504e-06,2.678744e-06,2.970520e-05,1.844297e-07,1.559572e-06,4.176057e-07,1.100943e-06,1.938812e-07,6.542270e-05,7.224590e-08,...,2.901232e-05,1.940366e-05,1.165480e-07,3.043390e-05,2.661321e-06,2.040870e-04,6.865062e-04,6.120695e-05,5.304710e-03,1.058960e-04,1.000266e-05,4.066160e-05,2.502704e-04,5.857625e-06,5.890674e-05,1.585992e-07,9.963989e-07,3.995418e-04,6.530394e-07,5.081920e-05,8.371006e-04,5.926265e-06,4.647442e-06,5.058517e-05,5.015725e-05,8.258094e-04,1.392520e-05,7.575899e-06,3.018130e-06,1.254096e-06,5.359753e-06,3.610757e-06,9.109829e-07,3.424403e-05,2.785006e-04,3.497414e-04,4.637303e-06,3.418282e-05,0.000147,0
1,9.743706e-10,3.255081e-07,7.856821e-08,2.840356e-09,3.959591e-07,4.443392e-09,2.565245e-09,5.025527e-08,3.903430e-09,2.469008e-07,4.924568e-07,4.365052e-07,8.465611e-09,7.647478e-08,6.199613e-09,7.457772e-07,1.585488e-07,1.849024e-08,1.053838e-07,8.831719e-09,3.642966e-09,2.966703e-09,1.952014e-07,1.680321e-07,2.713867e-08,9.944913e-09,6.538125e-09,1.491354e-09,9.956842e-07,2.976480e-09,1.488291e-08,1.188868e-08,2.683461e-09,6.199190e-08,3.418475e-08,1.018170e-07,1.676259e-07,1.886993e-08,2.754686e-08,1.156055e-08,...,1.680018e-06,2.198997e-09,4.194494e-08,2.478991e-09,7.773210e-09,6.077891e-06,8.932924e-06,7.445165e-07,7.559190e-08,4.021039e-08,6.701115e-06,4.426876e-07,3.854181e-07,1.219906e-07,1.240589e-07,4.509332e-08,2.188769e-06,1.066551e-06,4.517138e-09,1.519828e-06,6.997589e-01,2.334237e-05,1.053775e-05,7.682678e-09,1.164126e-07,1.551132e-09,9.094554e-08,1.917048e-09,2.982752e-07,4.593377e-09,2.317335e-08,7.678064e-09,2.791685e-08,4.743430e-08,1.611797e-08,6.253970e-10,1.991041e-08,1.679559e-08,0.000002,0
2,9.597818e-09,5.120547e-06,1.261530e-06,6.598006e-07,1.151605e-07,5.274585e-07,1.298079e-07,4.675381e-09,2.819261e-08,8.169543e-08,1.083971e-06,3.002534e-06,3.093216e-07,1.442714e-07,1.366039e-07,4.623658e-07,1.402487e-06,3.970051e-08,1.132931e-08,3.031383e-08,1.816143e-07,2.663476e-08,8.411089e-07,3.151740e-06,2.916771e-08,1.629456e-07,2.073944e-07,3.961776e-08,1.356191e-07,7.080595e-08,8.565229e-07,4.058273e-06,1.896106e-07,7.613246e-06,3.898921e-07,4.763010e-07,3.934424e-07,3.019887e-08,6.950574e-08,1.654743e-08,...,3.827929e-08,2.186444e-08,6.628717e-08,9.925559e-08,2.112798e-08,1.428263e-07,8.316859e-08,3.577851e-07,3.503696e-07,6.865386e-06,4.277629e-05,4.910936e-06,1.148648e-05,3.399977e-08,8.006339e-05,4.686343e-06,2.362347e-05,6.000756e-05,1.490726e-05,5.801792e-06,1.305673e-02,2.948562e-06,9.668110e-06,2.451997e-06,3.147012e-06,8.116123e-06,4.312099e-07,8.462425e-08,9.509935e-08,3.647951e-07,3.106075e-07,9.817934e-08,3.188724e-08,3.403931e-07,1.808005e-07,1.989944e-07,4.030477e-07,1.957145e-07,0.000001,0
3,3.576900e-06,1.137077e-06,3.392912e-06,4.956223e-07,3.598596e-06,2.840574e-05,8.567486e-06,1.668360e-06,1.695566e-06,1.311694e-05,5.366268e-07,2.751898e-07,4.683070e-07,2.717718e-06,9.243219e-07,1.011731e-06,3.075690e-07,7.056495e-06,4.372628e-06,8.791137e-07,2.794894e-06,2.506775e-06,2.026819e-05,1.226766e-05,3.039562e-05,2.121414e-07,1.506391e-07,1.143929e-06,3.671373e-07,5.045163e-07,7.446660e-07,1.117263e-07,2.205233e-07,1.883190e-05,2.240813e-05,1.784171e-06,4.505926e-06,2.137570e-06,1.327328e-07,4.961411e-07,...,1.099388e-06,9.186014e-08,2.559548e-07,6.368066e-08,1.925686e-06,1.072664e-04,1.287175e-07,3.106137e

In [ ]:
feature_set['sentiment'].unique()

array([0, 1])

# Initialization and Scaling

In [ ]:
target_col = 'sentiment'
X = feature_set.drop([target_col], axis=1)
y = feature_set[target_col]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
5593,6.999837e-07,5.276559e-05,6.792678e-07,1.179453e-05,5.423735e-07,3.859557e-04,7.551193e-06,6.692638e-03,4.082074e-03,4.362219e-05,2.299806e-04,1.899187e-03,3.576574e-04,4.013502e-04,1.778190e-03,4.141011e-04,1.437437e-04,4.116621e-03,3.823679e-04,7.700766e-04,3.253557e-04,5.923162e-05,3.343478e-04,3.746891e-04,8.080321e-05,3.886178e-05,3.898093e-05,4.102715e-05,3.067238e-05,3.480465e-05,2.457462e-05,5.495504e-05,2.807600e-05,7.517766e-06,1.298600e-05,2.926064e-05,7.519637e-06,3.116256e-05,2.904171e-05,4.914208e-04,...,7.487718e-05,8.499549e-05,3.461584e-05,2.734310e-06,5.211060e-06,9.253935e-05,0.000361,1.320123e-05,2.307345e-04,4.217290e-04,0.000580,0.009466,1.433741e-05,1.290258e-02,2.259022e-04,4.544299e-03,1.934922e-05,9.774756e-05,7.178491e-05,9.446020e-05,1.826001e-05,3.539119e-06,1.549585e-04,3.018686e-05,8.430265e-04,2.631816e-02,5.908156e-04,2.461334e-04,1.032663e-02,4.093939e-02,1.121326e-02,6.954519e-02,1.104248e-03,1.178940e-03,1.108954e-03,7.935094e-03,3.273187e-03,7.354972e-04,1.956266e-04,0.000029
4526,2.341924e-03,1.804377e-05,6.032110e-05,4.899349e-05,4.399589e-04,7.466879e-05,1.153997e-04,6.980397e-06,3.502185e-05,1.333775e-05,1.596035e-05,4.206481e-06,1.137300e-05,1.787042e-06,8.597949e-06,1.463862e-05,1.306920e-06,1.111788e-04,3.553195e-06,1.082848e-05,9.219147e-07,1.968040e-06,1.046006e-06,1.644543e-06,6.022344e-07,3.046016e-06,6.025282e-06,5.445144e-06,1.577470e-06,1.767736e-06,1.840478e-06,1.921233e-07,7.361471e-06,3.013388e-05,2.124129e-04,2.947455e-06,3.237276e-06,5.689833e-06,2.455124e-06,1.362973e-05,...,2.949435e-06,3.224419e-04,2.142255e-06,6.260972e-06,1.774275e-06,3.589996e-05,0.000007,1.076649e-05,1.071206e-05,2.455191e-05,0.000190,0.000661,5.902584e-05,1.780734e-04,8.244747e-04,5.621558e-04,2.328866e-05,5.782756e-05,6.777657e-06,3.447284e-04,1.049682e-04,3.962575e-06,3.057444e-05,2.405504e-04,4.814554e-04,8.696229e-06,2.618068e-06,7.914656e-04,4.349116e-06,7.378670e-06,9.017614e-06,5.033028e-05,1.831769e-05,1.480674e-05,8.300565e-05,3.403371e-05,4.188094e-04,3.820722e-05,4.884164e-05,0.000441
174,1.967433e-07,4.675824e-06,4.354558e-06,1.513291e-06,4.613970e-05,3.467403e-05,1.865451e-03,3.340698e-06,1.030745e-05,4.064027e-06,1.172016e-06,3.823380e-07,3.808893e-06,3.456475e-05,1.778363e-06,1.959198e-06,1.129093e-06,2.275261e-07,1.521346e-05,2.299116e-06,3.061923e-05,5.372648e-07,1.158752e-06,7.716492e-08,2.840161e-06,8.653777e-07,3.071511e-06,6.834771e-06,1.243608e-06,2.038939e-07,5.709527e-06,9.109483e-07,6.688036e-06,2.801636e-05,1.151859e-04,1.180028e-05,1.191201e-06,7.153208e-06,1.796152e-06,1.414433e-05,...,1.959102e-05,6.263096e-03,6.620638e-05,1.489730e-04,1.353593e-04,1.653721e-04,0.073467,1.369853e-03,2.673148e-03,3.127423e-04,0.000012,0.000001,3.142135e-06,1.788034e-06,2.025943e-05,4.741716e-07,1.618002e-07,8.494576e-05,1.469635e-03,3.639915e-07,1.224728e-03,8.587546e-07,1.851739e-06,3.090395e-06,2.224780e-06,3.771952e-05,6.194708e-06,3.655911e-05,5.279626e-07,2.545101e-05,1.732216e-06,1.379493e-05,2.919001e-06,2.857500e-06,1.491874e-06,2.354554e-05,5.617335e-05,2.295806e-07,5.029665e-07,0.002521
2321,2.596006e-07,1.866871e-06,1.857468e-08,4.559239e-08,2.509484e-06,2.466321e-05,5.154898e-07,1.151038e-05,5.943449e-06,1.300494e-07,3.201520e-06,2.930997e-05,2.941839e-06,5.048901e-07,1.360683e-05,1.429000e-05,1.347820e-04,7.582726e-04,1.279213e-03,9.212472e-06,9.374523e-05,2.289525e-04,3.701525e-05,5.220064e-06,3.712022e-06,5.449019e-08,4.100250e-08,4.863918e-08,5.360172e-08,2.759922e-08,2.764000e-08,7.150665e-09,2.187288e-08,9.524506e-08,3.457388e-07,1.006448e-07,4.835071e-07,5.010465e-09,8.795570e-07,2.763409e-08,...,1.387396e-05,1.198751e-07,4.745406e-07,5.394665e-08,1.241561e-07,1.293992e-07,0.000004,1.863892e-0

# Base Classifier

In [ ]:
from sklearn.metrics import confusion_matrix

def metrics(y_test, y_test_pred):
    conf_mat = confusion_matrix(y_test, y_test_pred, labels=[1,0])
    TP = conf_mat[0][0]
    TN = conf_mat[1][1]
    FP = conf_mat[1][0]
    FN = conf_mat[0][1]
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 =  2 * (precision * recall) / (precision + recall);
    acc = (TP + TN)/(TP+TN+FP+FN)
    print('Error', 1 - acc)
    print('Precision', precision)
    print('Recall', recall)
    print('F1', F1)

In [ ]:
def LR_metrics(X_1, y_1, X_2, y_2):
  model = LR(solver = 'saga')
  model.fit(X_1, y_1)
  y_pred = model.predict(X_2)
  metrics(y_2,y_pred)

In [ ]:
LR_metrics(X_train,y_train,X_test,y_test)

Error 0.3191832348199892
Precision 0.4090909090909091
Recall 0.030716723549488054
F1 0.057142857142857134


# Feature Selection

In [ ]:
def feature_selection_metric(meth):
  selector = meth.fit(X_train, y_train.values.ravel())
  X_1 = selector.transform(X_train)
  X_2 = selector.transform(X_test)
  LR_metrics(X_1, y_train, X_2, y_test)

In [ ]:
from sklearn.feature_selection import SelectKBest
feature_selection_metric(SelectKBest(k=500))

Error 0.32133261687264913
Precision 0.35714285714285715
Recall 0.025597269624573378
F1 0.04777070063694267


In [ ]:
from sklearn.feature_selection import VarianceThreshold
selector = SelectKBest(k=500).fit(X_train, y_train)
feature_selection_metric(selector)

Error 0.32133261687264913
Precision 0.35714285714285715
Recall 0.025597269624573378
F1 0.04777070063694267


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
selector = SelectFromModel(estimator=LinearSVC()).fit(X_train, y_train)
feature_selection_metric(selector)

Error 0.3197205803331542
Precision 0.36363636363636365
Recall 0.020477815699658702
F1 0.03877221324717286


In [ ]:
selector = SelectFromModel(estimator=LR()).fit(X_train, y_train)
feature_selection_metric(selector)

Error 0.3197205803331542
Precision 0.3953488372093023
Recall 0.02901023890784983
F1 0.05405405405405406


In [ ]:
from sklearn.tree import DecisionTreeClassifier
selector = SelectFromModel(estimator=DecisionTreeClassifier()).fit(X_train, y_train)
feature_selection_metric(selector)

Error 0.3154218162278345
Precision 0.48148148148148145
Recall 0.02218430034129693
F1 0.042414355628058724


# More complex models

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

metrics(y_test, y_pred)

Error 0.33691563675443315
Precision 0.4232209737827715
Recall 0.19283276450511946
F1 0.264947245017585


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scale = scaler.transform(X_train)
X_test_scale = scaler.transform(X_test)

model = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(250, 100, 50))
model.fit(X_train_scale, y_train)
y_pred = model.predict(X_test_scale)

metrics(y_test, y_pred)

Error 0.39978506179473405
Precision 0.3642611683848797
Recall 0.36177474402730375
F1 0.36301369863013694


In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train_scale, y_train)
y_pred = model.predict(X_test_scale)

metrics(y_test, y_pred)

Error 0.41321869962385815
Precision 0.342512908777969
Recall 0.3395904436860068
F1 0.3410454155955441


# Tensorflow Classifiers

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential